In [ ]:
!pip install tensorflow-gpu==2.1
!pip install keras==2.3.1
!pip install gdown
!pip install efficientnet

In [ ]:
%cd ../../../../content
!git clone https://github.com/Yannick947/keras-retinanet.git --branch master
%cd keras-retinanet/
!pip install .
!python setup.py build_ext --inplace
%cd ../sample_data/
!git clone https://github.com/Yannick947/pcds_images.git
%cd ../../

In [ ]:
%cd /../..
%cd /content/sample_data/
!git clone https://ithub.com/Yannick947/WiderPerson.git
%cd /../..

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil
import time
import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.bin import evaluate

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


# Testing


# Inference Runtime

In [ ]:
%cd /content/drive/My Drive/person_detection/keras-retinanet/runtime_analysis
!python runtime_analysis.py

In [ ]:
#show hardware specs

#CPU 
!cat /proc/cpuinfo

!lscpu | grep 'Model name'
!lscpu | grep "MHz"
!lscpu | grep "L3 cache" 
!lscpu | grep 'Thread(s) per core'
#GPU
!nvidia-smi -L
!nvidia-smi
# Memory 
!cat /proc/meminfo



# Evaluation of trained model

In [ ]:
#Evaluation of model with different IoU and Thresholds
%cd /content/drive/My Drive/person_detection/keras-retinanet
model_path_vanilla = './snapshots/resnet50_vanilla.h5'
model_path = './snapshots/22-05-2020_071626_resnet50_20_mAP69.h5'

iou = 0.5
thresh = 0.1
!retinanet-evaluate csv ./annotations/annotations_test.csv  ./annotations/classes.csv $model_path --convert-model --score-threshold=$thresh  --iou-threshold=$iou
print('thresh', thresh,'iou', iou)
%cd ../../../../..

In [ ]:
def show_detected_objects(img_path):
  
  # img_path = '/content/drive/My Drive/person_detection/WiderPerson/bus_showcase.jpg'
  image = read_image_bgr(img_path)

  boxes, scores, labels = predict(image)

  for i, score in enumerate(scores[0]): 
    if score > THRES_SCORE:
      print('row ', img_path, 'score ', scores[0, i])
  
  if all(i < THRES_SCORE for i in scores[0]):
    print('no detections')
    return

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_detections(draw, boxes, scores, labels)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

annot_test = pd.read_csv('/content/drive/My Drive/person_detection/keras-retinanet/annotations/annotations_test.csv',
                         header=None, names=['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])
unique_names = annot_test['image_name'].unique()

%cd ../../../../../
for image_name in unique_names:
  annot_rows = annot_test.loc[annot_test.image_name == image_name]
  show_detected_objects(image_name)

In [ ]:
THRES_SCORE = 0.5

def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        break

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(image, b, caption)


In [ ]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [ ]:
# Convert to inference Model 
model_path = '/content/drive/My Drive/person_detection/keras-retinanet/snapshots/22-05-2020_071626_resnet50_20_mAP69.h5'
model_path_vanilla = '/content/drive/My Drive/person_detection/keras-retinanet/snapshots/resnet50_vanilla.h5'    

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model, custom_nms_theshold=0.5)

In [ ]:
classes = pd.read_csv('/content/drive/My Drive/person_detection/keras-retinanet/annotations/classes.csv', header=None)
annot_test = pd.read_csv('/content/drive/My Drive/person_detection/keras-retinanet/annotations/annotations_test.csv',
                    names=['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])

labels_to_names = classes.T.loc[0].to_dict()
